# Загрузка данных и обновление существующих до версии 0.1.2

In [2]:
import random
import time
from datasets.pubchem import pubchem as pc
import pandas as pd

In [3]:
toxicity_data = pd.read_csv(r'../data/raw/pubchem/toxicity_data.csv')

In [4]:
start = 0
stop = 118040 # это максимальный номер в скачанном файле

# version 0.1.1 - если было 404 по версии 0.1.0, то повторный запрос всей секции Toxicity и сохранение всего отвеа
# version 0.1.2 - разбр всей секции Toxicity и сохранение найденного списка строк

for index, row in toxicity_data.loc[start:stop].iterrows():
    print(f"{index}/{stop}: cid={row['cid']}, code={row['status_code']}, version={row['version']} ==========")
    if row['status_code'] == 200: # повторные проходы
        if row['version'] in ['0.1.0', '0.1.2']:
            print("  Повтор. Code=200, v0.1.0/0.1.2 ... continue")
            continue
        elif row['version'] == '0.1.1': # разбор поля Toxicity_data_info и повышение версии строки
            print("  Повтор. Code=200, v0.1.1 ... upgrading to 0.1.2")
            r = eval(toxicity_data.at[index, 'toxicity_data_info'])
            toxicity_data.at[index, 'toxicity_data_value'] = pc.find_string_elements(r)
            toxicity_data.at[index, 'version'] = '0.1.2'
    else:
        try:
            print(f"  Loading new data for CID {row['cid']}")
            res = pc.get_toxicity_data(row['cid'], full_toxicity_data=True)
            time.sleep(random.random()/4+0.2)
            if res.is_success:
                toxicity_data.at[index, 'status_code'] = int(res.status_code)
                toxicity_data.at[index, 'toxicity_data_info'] = res.json()
                toxicity_data.at[index, 'toxicity_data_value'] = pc.find_string_elements(res.json())
                toxicity_data.at[index, 'version'] = '0.1.2'
            print(f"DONE: {index}/{stop}: cid={row['cid']}, {res.status_code}")
        except Exception as e:
            toxicity_data.at[index, 'status_code'] = 0
            toxicity_data.at[index, 'toxicity_data_info'] = None
            toxicity_data.at[index, 'toxicity_data_value'] = None
            print(f"ERROR: {index}/{stop}: cid={row['cid']}: {e}")
    if index % 10000 == 0:
        toxicity_data.to_csv(f'../data/raw/pubchem/toxicity_data_index-0.1.2-{index}.csv', index=False)
    

0/118040: cid=4, code=200.0, version=0.1.1 ==========
  Повтор. Code=200, v0.1.1 ... upgrading to 0.1.2
1/118040: cid=6, code=200.0, version=0.1.1 ==========
  Повтор. Code=200, v0.1.1 ... upgrading to 0.1.2
2/118040: cid=11, code=200.0, version=0.1.0 ==========
  Повтор. Code=200, v0.1.0/0.1.2 ... continue
3/118040: cid=13, code=200.0, version=0.1.0 ==========
  Повтор. Code=200, v0.1.0/0.1.2 ... continue
4/118040: cid=19, code=200.0, version=0.1.1 ==========
  Повтор. Code=200, v0.1.1 ... upgrading to 0.1.2
5/118040: cid=33, code=200.0, version=0.1.0 ==========
  Повтор. Code=200, v0.1.0/0.1.2 ... continue
6/118040: cid=34, code=200.0, version=0.1.0 ==========
  Повтор. Code=200, v0.1.0/0.1.2 ... continue
7/118040: cid=47, code=200.0, version=0.1.1 ==========
  Повтор. Code=200, v0.1.1 ... upgrading to 0.1.2
8/118040: cid=49, code=200.0, version=0.1.1 ==========
  Повтор. Code=200, v0.1.1 ... upgrading to 0.1.2
9/118040: cid=66, code=200.0, version=0.1.0 ==========
  Повтор. Code=200

KeyboardInterrupt: 

In [14]:
toxicity_data.to_csv(f'../data/raw/pubchem/toxicity_data.csv', index=False)